# Elementos esenciales

Alberga la base del sistema y la conexión.

## Librerías

Importamos las librerías necesarias para el funcionamiento del código.

In [1]:
import oracledb
import os 
import datetime
from pathlib import Path
from datetime import datetime
from typing import Optional
from dotenv import load_dotenv

## Conexión reutilizable

Se crea una función para conectarse a la base de datos local.

In [2]:
def get_connection():
    # Limpiar variables de entorno previas
    for key in ['USER', 'PASSWORD', 'DSN']:
        if key in os.environ:
            del os.environ[key]
    
    # Cargar variables de entorno desde el archivo .env
    env_path = r"c:\Users\vario\Documents\GitHub\Coeficiente-4\.env"
    load_dotenv(env_path, override=True)
    
    user = os.getenv("USER", "").strip().strip('"').strip("'").strip(',')
    password = os.getenv("PASSWORD", "").strip().strip('"').strip("'").strip(',')
    dsn = os.getenv("DSN", "").strip().strip('"').strip("'").strip(',')
    
    return oracledb.connect(
        user=user,
        password=password,
        dsn=dsn
    )

## Nota: Si recibe error ORA-28000 (cuenta bloqueada)

Ejecute en SQL*Plus o SQL Developer como administrador:
```sql
CONNECT SYS AS SYSDBA;
ALTER USER SYSTEM ACCOUNT UNLOCK;
ALTER USER SYSTEM IDENTIFIED BY 96443804;
```

Luego reinicie el servicio Oracle XE y vuelva a intentar.

# Creación de la base de datos

Código necesario para crear las tablas que representan las entidades de nuestro modelo.

## Esquema base de datos

Se define una función para crear todas las tablas a partir de una serie de consultas.

In [3]:
def create_schema(query):
    try:         
        with get_connection() as conn:             
            with conn.cursor() as cur:                 
                cur.execute(query)                 
                print(f'✓ Tabla creada exitosamente')
            conn.commit()     
    except oracledb.DatabaseError as e:        
        err = e
        if 'ORA-00955' in str(err):  # Tabla ya existe
            pass  # No imprimir nada si la tabla ya existe
        else:
            print(f"Error al crear tabla: {err}")

## Tablas de la base de datos

Se crea una función que alberga las consultas necesarias para la creación de las tablas y que llama la función para crear las tablas.

In [4]:
def create_all_tables():
    tables = [
        (
            "CREATE TABLE USUARIO ("
            "RUT VARCHAR(15) PRIMARY KEY,"
            "NOMBRES VARCHAR(64),"
            "APELLIDOS VARCHAR(64),"
            "EMAIL VARCHAR(64),"
            "CONTRASENA VARCHAR(32),"
            "CELULAR VARCHAR(15))"
        ),
        (
            "CREATE TABLE HORARIO ("
            "ID INTEGER PRIMARY KEY,"
            "HORA_INICIO TIMESTAMP,"
            "HORA_TERMINO TIMESTAMP,"
            "DISPONIBILIDAD CHAR(1))"
        ),
        (
            "CREATE TABLE CANCHA ("
            "ID INTEGER PRIMARY KEY,"
            "HORARIOFK INTEGER,"
            "TIPO VARCHAR(20),"
            "TARIFA INTEGER,"
            "FOREIGN KEY (HORARIOFK) REFERENCES HORARIO(ID))"
        ),
        (
            "CREATE TABLE PAGO ("
            "ID INTEGER PRIMARY KEY,"
            "USUARIOFK VARCHAR(15),"
            "MEDIO_PAGO VARCHAR(30),"
            "MONTO INTEGER,"
            "FOREIGN KEY (USUARIOFK) REFERENCES USUARIO(RUT))"
        ),
        (
            "CREATE TABLE RESERVA ("
            "ID INTEGER PRIMARY KEY,"
            "HORARIOFK INTEGER,"
            "CANCHAFK INTEGER,"
            "USUARIOFK VARCHAR(15),"
            "PAGOFK INTEGER,"
            "ESTADO CHAR(1),"
            "FOREIGN KEY (HORARIOFK) REFERENCES HORARIO(ID),"
            "FOREIGN KEY (CANCHAFK) REFERENCES CANCHA(ID),"
            "FOREIGN KEY (USUARIOFK) REFERENCES USUARIO(RUT),"
            "FOREIGN KEY (PAGOFK) REFERENCES PAGO(ID))"
        )
    ]
    for query in tables:
        create_schema(query)

## Función para limpiar tablas previamente creadas

Para agilizar el proceso de testeo y demostración, se creó una función que limpiará la base de datos si es que esta estaba previamente poblada.

In [5]:
def drop_all_tables():
    try:
        # Use context managers so connections and cursors are closed automatically
        with get_connection() as conn:
            with conn.cursor() as cur:
                # Orden de eliminación (respetando foreign keys)
                tablas = ['RESERVA', 'PAGO', 'CANCHA', 'HORARIO', 'USUARIO']

                for tabla in tablas:
                    try:
                        cur.execute(f'DROP TABLE {tabla}')
                        conn.commit()
                        print(f'✓ Tabla {tabla} eliminada')
                    except oracledb.DatabaseError as e:
                        if 'ORA-00942' in str(e):  # Tabla no existe
                            print(f'- Tabla {tabla} no existe (ignorado)')
                        else:
                            print(f'✗ Error al eliminar {tabla}: {e}')

        print("\n✓ Limpieza completada exitosamente")

    except oracledb.DatabaseError as e:
        print(f"Error de conexión: {e}")

# Funciones para insertar datos

Debido a que la estructura de cada tabla varía, es necesario crear funciones específicas para insertar datos en la entidad correspondiente.

## Tabla Usuario

In [6]:
def create_usuario(RUT, NOMBRES, APELLIDOS, EMAIL, CONTRASENA, CELULAR):     
    sql = (        
    "INSERT INTO USUARIO (RUT, NOMBRES, APELLIDOS, EMAIL, CONTRASENA, CELULAR) "         
    "VALUES (:RUT, :NOMBRES, :APELLIDOS, :EMAIL, :CONTRASENA, :CELULAR)"     
    )     
    try:
        with get_connection() as conn:         
            with conn.cursor() as cur:             
                cur.execute(sql, {                 
                "RUT": RUT,                 
                "NOMBRES": NOMBRES,                 
                "APELLIDOS": APELLIDOS,
                "EMAIL": EMAIL,
                "CONTRASENA": CONTRASENA,
                "CELULAR": CELULAR           
                })             
            conn.commit()            
        print(f"Usuario con RUT={RUT} creado.")
    except oracledb.DatabaseError as e:
        print(f"Error al crear usuario: {e}")

## Tabla Horario

In [7]:
def create_horario(ID, HORA_INICIO, HORA_TERMINO, DISPONIBILIDAD):     
    sql = (        
    "INSERT INTO HORARIO (ID, HORA_INICIO, HORA_TERMINO, DISPONIBILIDAD) "         
    "VALUES (:ID, :HORA_INICIO, :HORA_TERMINO, :DISPONIBILIDAD)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,
            "HORA_INICIO": HORA_INICIO,
            "HORA_TERMINO": HORA_TERMINO,
            "DISPONIBILIDAD": DISPONIBILIDAD
            })             
        conn.commit()            
    print(f"Horario {ID} creado.")

## Tabla Cancha

In [8]:
def create_cancha(ID, HORARIOFK, TIPO, TARIFA):     
    sql = (        
    "INSERT INTO CANCHA (ID, HORARIOFK, TIPO, TARIFA) "         
    "VALUES (:ID, :HORARIOFK, :TIPO, :TARIFA)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,
            "HORARIOFK": HORARIOFK,
            "TIPO": TIPO,
            "TARIFA": TARIFA
            })             
        conn.commit()            
    print(f"Cancha {ID} registrada.")

## Tabla Pagos

In [9]:
def create_pago(ID, USUARIOFK, MEDIO_PAGO, MONTO):     
    sql = (        
    "INSERT INTO PAGO (ID, USUARIOFK, MEDIO_PAGO, MONTO) "         
    "VALUES (:ID, :USUARIOFK, :MEDIO_PAGO, :MONTO)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,                 
            "USUARIOFK": USUARIOFK,
            "MEDIO_PAGO": MEDIO_PAGO,
            "MONTO": MONTO
            })             
        conn.commit()            
    print(f"Usuario {USUARIOFK} ha realizado el pago {ID}.")

## Tabla Reserva

In [10]:
def create_reserva(ID, HORARIOFK, CANCHAFK, USUARIOFK, PAGOFK, ESTADO):     
    sql = (        
    "INSERT INTO RESERVA (ID, HORARIOFK, CANCHAFK, USUARIOFK, PAGOFK, ESTADO) "         
    "VALUES (:ID, :HORARIOFK, :CANCHAFK, :USUARIOFK, :PAGOFK, :ESTADO)"     
    )    
    with get_connection() as conn:         
        with conn.cursor() as cur:             
            cur.execute(sql, {                 
            "ID": ID,                 
            "HORARIOFK": HORARIOFK,
            "CANCHAFK": CANCHAFK,
            "USUARIOFK": USUARIOFK,
            "PAGOFK": PAGOFK,
            "ESTADO": ESTADO
            })             
        conn.commit()            
    print(f"Reserva {ID} creada para usuario {USUARIOFK}.")

## Poblamiento de tablas

In [11]:
drop_all_tables()
create_all_tables()

✓ Tabla RESERVA eliminada
✓ Tabla PAGO eliminada
✓ Tabla CANCHA eliminada
✓ Tabla HORARIO eliminada
✓ Tabla USUARIO eliminada

✓ Limpieza completada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente
✓ Tabla creada exitosamente


### Poblar tabla Usuario

In [12]:
create_usuario("20.123.456-7", "Juan", "Pérez", "juan@email.com", "pass123", "987654321")
create_usuario("19.987.654-3", "María", "García", "maria@email.com", "pass456", "976543210")

Usuario con RUT=20.123.456-7 creado.
Usuario con RUT=19.987.654-3 creado.


### Poblar tabla Horario

In [13]:
create_horario(1, datetime.now(), datetime.now(), True)
create_horario(2, datetime.now(), datetime.now(), True)
create_horario(3, datetime.now(), datetime.now(), False)

Horario 1 creado.
Horario 2 creado.
Horario 3 creado.


### Poblar tabla Cancha

In [14]:
create_cancha(1, 1, "Pasto", 50000)
create_cancha(2, 2, "Cemento", 40000)
create_cancha(3, 3, "Sintética", 55000)

Cancha 1 registrada.
Cancha 2 registrada.
Cancha 3 registrada.


### Poblar tabla Pago

In [15]:
create_pago(1, "20.123.456-7", "Tarjeta Crédito", 50000)
create_pago(2, "19.987.654-3", "Transferencia Bancaria", 40000)
create_pago(3, "20.123.456-7", "Efectivo", 55000)

Usuario 20.123.456-7 ha realizado el pago 1.
Usuario 19.987.654-3 ha realizado el pago 2.
Usuario 20.123.456-7 ha realizado el pago 3.


### Poblar tabla Reserva

In [16]:
create_reserva(1, 1, 1, "20.123.456-7", 1, True)
create_reserva(2, 2, 2, "19.987.654-3", 2, True)
create_reserva(3, 3, 3, "20.123.456-7", 3, False)

Reserva 1 creada para usuario 20.123.456-7.
Reserva 2 creada para usuario 19.987.654-3.
Reserva 3 creada para usuario 20.123.456-7.


# Funciones para leer datos

Se definen funciones para obtener todos los datos de una tabla determinada o para buscar un elemento de una tabla determinada.

## Leer todos los datos de una tabla

In [17]:
def read_table(tabla):
    sql = (
        f"SELECT * FROM {tabla}"
    )
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                resultados = cur.execute(sql)
                print(f"Consulta a la tabla {tabla}")
                for fila in resultados:
                    print(fila)
    except oracledb.DatabaseError as e:
        err = e
        print(f"Error al leer datos: {err}")

## Buscar por identificador

In [18]:
def read_table_by_id(tabla, columna_id, valor_id):
    sql = (
        f"SELECT * FROM {tabla} WHERE {columna_id} = :valor"
    )
    parametros = {"valor": valor_id}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                resultados = cur.execute(sql, parametros)
                print(f"✓ Consulta a la tabla {tabla} donde {columna_id} = {valor_id}")
                encontrados = 0
                for fila in resultados:
                    print(f"  {fila}")
                    encontrados += 1
                if encontrados == 0:
                    print(f"  No se encontraron registros")
    except oracledb.DatabaseError as e:
        print(f"Error al leer datos: {e}")

# Funciones para actualizar datos

## Actualizar Usuario

In [19]:
def update_usuario(
    RUT,
    NOMBRES: str = None,
    APELLIDOS: Optional[str] = None,
    EMAIL: Optional[str] = None,
    CONTRASENA: Optional[str] = None,
    CELULAR: Optional[str] = None
):
    modificaciones = []
    parametros = {"RUT": RUT}

    if RUT is not None:
        modificaciones.append("RUT =: RUT")
        parametros["RUT"] = RUT
    if NOMBRES is not None:
        modificaciones.append("NOMBRES =: NOMBRES")
        parametros["NOMBRES"] = NOMBRES
    if APELLIDOS is not None:
        modificaciones.append("APELLIDOS =: APELLIDOS")
        parametros["APELLIDOS"] = APELLIDOS
    if EMAIL is not None:
        modificaciones.append("EMAIL =: EMAIL")
        parametros["EMAIL"] = EMAIL
    if CONTRASENA is not None:
        modificaciones.append("CONTRASENA =: CONTRASENA")
        parametros["CONTRASENA"] = CONTRASENA
    if CELULAR is not None:
        modificaciones.append("CELULAR =: CELULAR")
        parametros["CELULAR"] = CELULAR
    if not modificaciones:
        return print("No hay campos para actualizar.")

    sql = f"UPDATE USUARIOS SET {', '.join(modificaciones)} WHERE RUT = :RUT"

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
        print(f"Usuario con RUT={RUT} actualizada.")

## Actualizar Horario

In [20]:
def update_horario(
    ID,
    HORA_INICIO: Optional[datetime] = None,
    HORA_TERMINO: Optional[datetime] = None,
    DISPONIBILIDAD: Optional[bool] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if HORA_INICIO is not None:
        modificaciones.append("HORA_INICIO =: HORA_INICIO")
        parametros["HORA_INICIO"] = HORA_INICIO
    if HORA_TERMINO is not None:
        modificaciones.append("HORA_TERMINO =: HORA_TERMINO")
        parametros["HORA_TERMINO"] = HORA_TERMINO
    if DISPONIBILIDAD is not None:
        modificaciones.append("DISPONIBILIDAD =: DISPONIBILIDAD")
        parametros["DISPONIBILIDAD"] = DISPONIBILIDAD
    if not modificaciones:
        return print("No hay campos para actualizar.")

    sql = f"UPDATE HORARIO SET {', '.join(modificaciones)} WHERE ID = :ID"

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
        print(f"Horario con ID={ID} actualizado.")

## Actualizar Cancha

In [21]:
def update_cancha(
    ID,
    HORARIOFK: Optional[int] = None,
    TIPO: Optional[str] = None,
    TARIFA: Optional[int] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if HORARIOFK is not None:
        modificaciones.append("HORARIOFK =: HORARIOFK")
        parametros["HORARIOFK"] = HORARIOFK
    if TIPO is not None:
        modificaciones.append("TIPO =: TIPO")
        parametros["TIPO"] = TIPO
    if TARIFA is not None:
        modificaciones.append("TARIFA =: TARIFA")
        parametros["TARIFA"] = TARIFA
    if not modificaciones:
        return print("No hay campos para actualizar.")

    sql = f"UPDATE CANCHA SET {', '.join(modificaciones)} WHERE ID = :ID"

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
        print(f"Cancha con ID={ID} actualizada.")

## Actualizar Pago

In [22]:
def update_pagos(
    ID,
    USUARIOFK: Optional[str] = None,
    MEDIO_PAGO: Optional[str] = None,
    MONTO: Optional[int] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if USUARIOFK is not None:
        modificaciones.append("USUARIOFK =: USUARIOFK")
        parametros["USUARIOFK"] = USUARIOFK
    if MEDIO_PAGO is not None:
        modificaciones.append("MEDIO_PAGO =: MEDIO_PAGO")
        parametros["MEDIO_PAGO"] = MEDIO_PAGO
    if MONTO is not None:
        modificaciones.append("MONTO =: MONTO")
        parametros["MONTO"] = MONTO
    if not modificaciones:
        return print("No hay campos para actualizar.")

    sql = f"UPDATE PAGO SET {', '.join(modificaciones)} WHERE ID = :ID"

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
        print(f"Pago con ID={ID} actualizado.")

## Actualizar Reserva

In [23]:
def update_reserva(
    ID,
    HORARIOFK: Optional[int] = None,
    CANCHAFK: Optional[int] = None,
    USUARIOFK: Optional[str] = None,
    PAGOFK: Optional[int] = None,
    ESTADO: Optional[bool] = None
):
    modificaciones = []
    parametros = {"ID": ID}

    if HORARIOFK is not None:
        modificaciones.append("HORARIOFK =: HORARIOFK")
        parametros["HORARIOFK"] = HORARIOFK
    if CANCHAFK is not None:
        modificaciones.append("CANCHAFK =: CANCHAFK")
        parametros["CANCHAFK"] = CANCHAFK
    if USUARIOFK is not None:
        modificaciones.append("USUARIOFK =: USUARIOFK")
        parametros["USUARIOFK"] = USUARIOFK
    if PAGOFK is not None:
        modificaciones.append("PAGOFK =: PAGOFK")
        parametros["PAGOFK"] = PAGOFK
    if ESTADO is not None:
        modificaciones.append("ESTADO =: ESTADO")
        parametros["ESTADO"] = ESTADO
    if not modificaciones:
        return print("No hay campos para actualizar.")

    sql = f"UPDATE RESERVA SET {', '.join(modificaciones)} WHERE ID = :ID"

    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, parametros)
        conn.commit()
        print(f"Reserva con ID={ID} actualizada.")

# Funciones para eliminar datos

## Eliminar Usuario

In [24]:
def delete_usuario(RUT):
    sql = (
        "DELETE FROM USUARIO WHERE RUT = :RUT"
    )
    parametros = {"RUT" : RUT}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
            print(f"Usuario con RUT {RUT} eliminado.")
    except oracledb.DatabaseError as e:
        err = e
        print(f"Error al eliminar dato: {err} \n {sql} \n {parametros}")

## Eliminar Horario

In [25]:
def delete_horario(ID: int):
    sql = "DELETE FROM HORARIO WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Horario con ID {ID} eliminado.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar horario {ID}: {e}")

## Eliminar Cancha

In [26]:
def delete_cancha(ID: int):
    sql = "DELETE FROM CANCHA WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Cancha con ID {ID} eliminada.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar cancha {ID}: {e}")

## Eliminar Pago

In [27]:
def delete_pago(ID: str):
    sql = "DELETE FROM PAGO WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Pago con ID {ID} eliminado.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar pago {ID}: {e}")

## Eliminar Reserva

In [28]:
def delete_reserva(ID: int):
    sql = "DELETE FROM RESERVA WHERE ID = :ID"
    parametros = {"ID": ID}
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, parametros)
            conn.commit()
        print(f"Reserva con ID {ID} eliminada.")
    except oracledb.DatabaseError as e:
        print(f"Error al eliminar reserva {ID}: {e}")

# Menú de Usuario

In [29]:
def __main__():
    while True:
        print(
            """
            ==========================================
            |     Sistema de Reserva de Canchas      |
            ==========================================
            |    Bienvenido al sistema de gestión    |
            |          ¿Qué deseas hacer?:           |
            ==========================================
            | 1. Ingresar datos                      |
            | 2. Consultar datos                     |
            | 3. Actualizar datos                    |
            | 4. Eliminar datos                      |
            | 0. Salir                               |
            ==========================================
            """
        )
        opcion = input("Selecciona una opcion [1-4, 0 para salir]: ")
        
        if opcion == "0":
            print("Saliendo del sistema...")
            break
            
        elif opcion == "1":
            print(
                """
                ==========================================
                |     Sistema de Reserva de Canchas      |
                ==========================================
                |   ¿Dónde deseas ingresar los datos?    |
                ==========================================
                | 1. Tabla Usuario                       |
                | 2. Tabla Horario                       |
                | 3. Tabla Cancha                        |
                | 4. Tabla Pago                          |
                | 5. Tabla Reserva                       |
                | 0. Salir                               |
                ==========================================
                """
            )
            opcion_ingresar = input("Selecciona una opcion [1-5, 0 para salir]: ")
            if opcion_ingresar == "0":
                print("Regresando...\n")
            elif opcion_ingresar == "1":
                rut = input("Ingrese RUT: ")
                if not rut:
                    print("Error: RUT no puede estar vacío.\n")
                    continue
                nombres = input("Ingrese Nombres: ")
                if not nombres:
                    print("Error: Nombres no puede estar vacío.\n")
                    continue
                apellidos = input("Ingrese Apellidos: ")
                if not apellidos:
                    print("Error: Apellidos no puede estar vacío.\n")
                    continue
                email = input("Ingrese Email: ")
                if not email:
                    print("Error: Email no puede estar vacío.\n")
                    continue
                contrasena = input("Cree una Contraseña: ")
                if not contrasena:
                    print("Error: Contraseña no puede estar vacía.\n")
                    continue
                celular = input("Ingrese Celular: ")
                if not celular:
                    print("Error: Celular no puede estar vacío.\n")
                    continue
                create_usuario(rut, nombres, apellidos, email, contrasena, celular)
                print()
            elif opcion_ingresar == "2":
                try:
                    id = int(input("Ingrese ID horario: "))
                    hora_inicio = datetime.timestamp(input("Ingrese la hora de inicio: "))
                    hora_termino = datetime.timestamp(input("Ingrese la hora de término: "))
                    disponibilidad = input("¿Está disponible? (True/False): ").strip().lower()
                    create_horario(id, hora_inicio, hora_termino, disponibilidad)
                    print()
                except ValueError:
                    print("Error: Ingrese valores válidos (ID debe ser un número).\n")
            elif opcion_ingresar == "3":
                try:
                    id = int(input("Ingrese ID cancha: "))
                    horariofk = int(input("Ingrese ID horario asociado: "))
                    tipo = input("Ingrese Tipo de Cancha: ")
                    if not tipo:
                        print("Error: Tipo de cancha no puede estar vacío.\n")
                        continue
                    tarifa = int(input("Ingrese Tarifa de la Cancha: "))
                    create_cancha(id, horariofk, tipo, tarifa)
                    print()
                except ValueError:
                    print("Error: Ingrese valores válidos (ID y Tarifa deben ser números).\n")
            elif opcion_ingresar == "4":
                try:
                    id = int(input("Ingrese ID del pago: "))
                    usuariofk = input("Ingrese RUT del Usuario: ")
                    if not usuariofk:
                        print("Error: RUT no puede estar vacío.\n")
                        continue
                    medio_pago = input("Ingrese Medio de Pago: ")
                    if not medio_pago:
                        print("Error: Medio de pago no puede estar vacío.\n")
                        continue
                    monto = int(input("Ingrese Monto del Pago: "))
                    create_pago(id, usuariofk, medio_pago, monto)
                    print()
                except ValueError:
                    print("Error: Ingrese valores válidos (ID y Monto deben ser números).\n")
            elif opcion_ingresar == "5":
                try:
                    id = int(input("Ingrese ID de la reserva: "))
                    horariofk = int(input("Ingrese ID horario asociado: "))
                    canchafk = int(input("Ingrese ID de la cancha asociada: "))
                    usuariofk = input("Ingrese RUT del Usuario: ")
                    if not usuariofk:
                        print("Error: RUT no puede estar vacío.\n")
                        continue
                    pagofk = int(input("Ingrese ID del Pago asociado: "))
                    estado = input("¿Está activa la reserva? (True/False): ").strip().lower()
                    create_reserva(id, horariofk, canchafk, usuariofk, pagofk, estado)
                    print()
                except ValueError:
                    print("Error: Ingrese valores válidos (IDs deben ser números).\n")
            else:
                print("Opción inválida. Intente nuevamente.\n")

        elif opcion == "2":
            print(
                """
                ==========================================
                |     Sistema de Reserva de Canchas      |
                ==========================================
                |     ¿Cómo deseas consultar los datos?  |
                ==========================================
                | 1. Quiero consultar todos los datos    |
                |    de una tabla.                       |
                | 2. Quiero consultar un dato            |
                |    específico.                         |
                | 0. Salir                               |
                ==========================================
                """
            )
            opcion_consultar = input("Selecciona una opción [1-2, 0 para salir]: ")
            
            if opcion_consultar == "0":
                print("Regresando...\n")
            elif opcion_consultar == "1":
                print(
                    """
                    ==========================================
                    |     Sistema de Reserva de Canchas      |
                    ==========================================
                    |   ¿Cuál tabla deseas consultar?        |
                    ==========================================
                    | 1. Tabla Usuario                       |
                    | 2. Tabla Horario                       |
                    | 3. Tabla Cancha                        |
                    | 4. Tabla Pago                          |
                    | 5. Tabla Reserva                       |
                    | 0. Salir                               |
                    ==========================================
                    """
                )
                opcion_tabla = input("Selecciona una opción [1-5, 0 para salir]: ")
                if opcion_tabla == "0":
                    print("Regresando...\n")
                elif opcion_tabla == "1":
                    read_table("USUARIO")
                    print()
                elif opcion_tabla == "2":
                    read_table("HORARIO")
                    print()
                elif opcion_tabla == "3":
                    read_table("CANCHA")
                    print()
                elif opcion_tabla == "4":
                    read_table("PAGO")
                    print()
                elif opcion_tabla == "5":
                    read_table("RESERVA")
                    print()
                else:
                    print("Opción inválida. Intente nuevamente.\n")
            elif opcion_consultar == "2":
                tabla = input("¿Qué tabla deseas consultar?: [Usuario - Horario - Cancha - Pago - Reserva]: ").upper()
                if not tabla:
                    print("Error: Tabla no puede estar vacía.\n")
                else:
                    parametro = input("¿Por cuál parámetro buscarás el dato?: [Id - RUT - Nombre - ETC]: ").upper()
                    if not parametro:
                        print("Error: Parámetro no puede estar vacío.\n")
                    else:
                        valor = input("Ingresa el identificador: ").upper()
                        if not valor:
                            print("Error: Identificador no puede estar vacío.\n")
                        else:
                            read_table_by_id(tabla, parametro, valor)
                            print()
            else:
                print("Opción inválida. Intente nuevamente.\n")
        
        elif opcion == "3":
            print(
                """
                ==========================================
                |     Sistema de Reserva de Canchas      |
                ==========================================
                |   ¿Dónde deseas actualizar los datos?  |
                ==========================================
                | 1. Tabla Usuario                       |
                | 2. Tabla Horario                       |
                | 3. Tabla Cancha                        |
                | 4. Tabla Pago                          |
                | 5. Tabla Reserva                       |
                | 0. Salir                               |
                ==========================================
                """
            )
            opcion_actualizar = input("Selecciona una opcion [1-5, 0 para salir]: ")
            if opcion_actualizar == "0":
                print("Regresando...\n")
            elif opcion_actualizar == "1":
                rut = input("Ingrese RUT de la persona a actualizar: ")
                if not rut:
                    print("Error: RUT no puede estar vacío.\n")
                else:
                    nombres = input("Ingrese nuevos Nombres (dejar en blanco para no cambiar): ")
                    apellidos = input("Ingrese nuevos Apellidos (dejar en blanco para no cambiar): ")
                    email = input("Ingrese nuevo Email (dejar en blanco para no cambiar): ")
                    contrasena = input("Ingrese nueva Contraseña (dejar en blanco para no cambiar): ")
                    celular = input("Ingrese nuevo Celular (dejar en blanco para no cambiar): ")
                    update_usuario(
                        RUT=rut,
                        NOMBRES=nombres if nombres else None,
                        APELLIDOS=apellidos if apellidos else None,
                        EMAIL=email if email else None,
                        CONTRASENA=contrasena if contrasena else None,
                        CELULAR=celular if celular else None
                    )
                    print()
            elif opcion_actualizar == "2":
                try:
                    id = int(input("Ingrese ID del horario a actualizar: "))
                    hora_inicio = input("Ingrese nueva Hora de Inicio (dejar en blanco para no cambiar): ")
                    hora_termino = input("Ingrese nueva Hora de Término (dejar en blanco para no cambiar): ")
                    disponibilidad = input("Ingrese nueva Disponibilidad (True/False) (dejar en blanco para no cambiar): ")
                    update_horario(
                        ID=id,
                        HORA_INICIO=datetime.timestamp(hora_inicio) if hora_inicio else None,
                        HORA_TERMINO=datetime.timestamp(hora_termino) if hora_termino else None,
                        DISPONIBILIDAD=disponibilidad.strip().lower() == 'true' if disponibilidad else None
                    )
                    print()
                except ValueError:
                    print("Error: Ingrese valores válidos (ID debe ser un número).\n")
            elif opcion_actualizar == "3":
                try:
                    id = int(input("Ingrese ID de la cancha a actualizar: "))
                    horariofk = input("Ingrese nuevo ID del Horario (dejar en blanco para no cambiar): ")
                    tipo = input("Ingrese nuevo Tipo de Cancha (dejar en blanco para no cambiar): ")
                    tarifa = input("Ingrese nueva Tarifa (dejar en blanco para no cambiar): ")
                    update_cancha(
                        ID=id,
                        HORARIOFK=int(horariofk) if horariofk else None,
                        TIPO=tipo if tipo else None,
                        TARIFA=int(tarifa) if tarifa else None
                    )
                    print()
                except ValueError:
                    print("Error: Ingrese valores válidos (Horario e ID deben ser números).\n")
            elif opcion_actualizar == "4":
                try:
                    id = int(input("Ingrese ID del pago a actualizar: "))
                    usuariofk = input("Ingrese nuevo RUT del Usuario (dejar en blanco para no cambiar): ")
                    medio_pago = input("Ingrese nuevo Medio de Pago (dejar en blanco para no cambiar): ")
                    monto = input("Ingrese nuevo Monto (dejar en blanco para no cambiar): ")
                    update_pagos(
                        ID=id,
                        USUARIOFK=usuariofk if usuariofk else None,
                        MEDIO_PAGO=medio_pago if medio_pago else None,
                        MONTO=int(monto) if monto else None
                    )
                    print()
                except ValueError:
                    print("Error: Ingrese valores válidos (ID y Monto deben ser números).\n")
            elif opcion_actualizar == "5":
                try:
                    id = int(input("Ingrese ID de la reserva a actualizar: "))
                    horariofk = input("Ingrese nuevo ID del Horario (dejar en blanco para no cambiar): ")
                    canchafk = input("Ingrese nuevo ID de la Cancha (dejar en blanco para no cambiar): ")
                    usuariofk = input("Ingrese nuevo RUT del Usuario (dejar en blanco para no cambiar): ")
                    pagofk = input("Ingrese nuevo ID del Pago (dejar en blanco para no cambiar): ")
                    estado = input("Ingrese nuevo Estado (True/False) (dejar en blanco para no cambiar): ")
                    update_reserva(
                        ID=id,
                        HORARIOFK=int(horariofk) if horariofk else None,
                        CANCHAFK=int(canchafk) if canchafk else None,
                        USUARIOFK=usuariofk if usuariofk else None,
                        PAGOFK=int(pagofk) if pagofk else None,
                        ESTADO=estado.strip().lower() == 'true' if estado else None
                    )
                    print()
                except ValueError:
                    print("Error: Ingrese valores válidos (IDs deben ser números).\n")
            else:
                print("Opción inválida. Intente nuevamente.\n")

        elif opcion == "4":
            print(
                """
                ==========================================
                |     Sistema de Reserva de Canchas      |
                ==========================================
                |   ¿Dónde deseas eliminar los datos?    |
                ==========================================
                | 1. Tabla Usuario                       |
                | 2. Tabla Horario                       |
                | 3. Tabla Cancha                        |
                | 4. Tabla Pago                          |
                | 5. Tabla Reserva                       |
                | 0. Salir                               |
                ==========================================
                """
            )
            opcion_eliminar = input("Selecciona una opcion [1-5, 0 para salir]: ")
            if opcion_eliminar == "0":
                print("Regresando...\n")
            elif opcion_eliminar == "1":
                rut = input("Ingrese RUT del usuario a eliminar: ")
                if not rut:
                    print("Error: RUT no puede estar vacío.\n")
                else:
                    delete_usuario(rut)
                    print()
            elif opcion_eliminar == "2":
                try:
                    id = int(input("Ingrese ID del horario a eliminar: "))
                    delete_horario(id)
                    print()
                except ValueError:
                    print("Error: Ingrese un ID válido (debe ser un número).\n")
            elif opcion_eliminar == "3":
                try:
                    id = int(input("Ingrese ID de la cancha a eliminar: "))
                    delete_cancha(id)
                    print()
                except ValueError:
                    print("Error: Ingrese un ID válido (debe ser un número).\n")
            elif opcion_eliminar == "4":
                try:
                    id = int(input("Ingrese ID del pago a eliminar: "))
                    delete_pago(id)
                    print()
                except ValueError:
                    print("Error: Ingrese un ID válido (debe ser un número).\n")
            elif opcion_eliminar == "5":
                try:
                    id = int(input("Ingrese ID de la reserva a eliminar: "))
                    delete_reserva(id)
                    print()
                except ValueError:
                    print("Error: Ingrese un ID válido (debe ser un número).\n")
            else:
                print("Opción inválida. Intente nuevamente.\n")
        else:
            print("Opción inválida. Intente nuevamente.\n")

if __name__ == "__main__":
    __main__()


            |     Sistema de Reserva de Canchas      |
            |    Bienvenido al sistema de gestión    |
            |          ¿Qué deseas hacer?:           |
            | 1. Ingresar datos                      |
            | 2. Consultar datos                     |
            | 3. Actualizar datos                    |
            | 4. Eliminar datos                      |
            | 0. Salir                               |
            

                |     Sistema de Reserva de Canchas      |
                |     ¿Cómo deseas consultar los datos?  |
                | 1. Quiero consultar todos los datos    |
                |    de una tabla.                       |
                | 2. Quiero consultar un dato            |
                |    específico.                         |
                | 0. Salir                               |
                

                    |     Sistema de Reserva de Canchas      |
                    |   ¿Cuál tabla deseas consulta